In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import napari
from PIL import Image

from scribbles_creator import *
from cellpose_data_handler import *

c:\Users\roman\anaconda3\envs\ilastik2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Define where the images are located and what data to consider

In [2]:
folder_path = "./cellpose_train_imgs"

In [3]:
mode = "all"
bins = [0.1]#, 1, 1.25]#[0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1]
scribble_width = 2
all_suff = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"]
suff = ["a"] #all_suff[:5]
prediction_types = ["dino"] #["convpaint", "ilastik"] # ["convpaintCh1", "ilastikCh1"]

## Analyse the created scribbles and segmentations (perc. labelled vs. acc.) from saved files

In [4]:
df_list = []

for img_num in [1]: #range(0, 540, 10):
    for bin in bins:
        for s in suff:
            for pred in prediction_types:
                res = analyse_cellpose_single_file(folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s, pred_tag=pred, show_res=False)
                df_list.append(res)

df = pd.concat(df_list, ignore_index=True)
df[:5]                

,img_num,prediction type,scribbles mode,scribbles bin,suffix,class_1_pix_gt,class_2_pix_gt,pix_labelled,class_1_pix_labelled,class_2_pix_labelled,...,perc. labelled,accuracy,mPrecision,mRecall,mIoU,mF1,image,ground truth,scribbles,prediction
0,1,dino,all,0.1,a,128463,67633,145,100,45,...,0.073943,0.618529,0.554094,0.544647,0.395206,0.542738,./cellpose_train_imgs/001_img.png,./cellpose_train_imgs/001_ground_truth.png,./cellpose_train_imgs/001_scribbles_all_00100_...,./cellpose_train_imgs/001_dino_all_00100_w2_a.png


Analyse the comparison of convpaint models

In [10]:
folder_path = "cellpose_train_imgs"
prediction_types = ["convpaint_l-0_s-1",
                    "convpaint_l-0_s-1-2",
                    "convpaint_l-0_s-1-2-4",
                    "convpaint_l-0_s-1-2-4-8",
                    "convpaint_l-0-2_s-1",
                    "convpaint_l-0-2_s-1-2",
                    "convpaint_l-0-2_s-1-2-4",
                    "convpaint_l-0-2_s-1-2-4-8"] # , "ilastik", "dino"] # ["convpaintCh1", "ilastikCh1"]

df = pd.DataFrame(columns=['img_num', 'prediction type', 'scribbles mode', 'scribbles bin', 'suffix', 'class_1_pix_gt', 'class_2_pix_gt', 'pix_labelled', 'class_1_pix_labelled', 'class_2_pix_labelled', 'pix_in_img', 'perc. labelled', 'accuracy', 'image', 'ground truth', 'scribbles', 'prediction'])
df_list = []

for img_num in [1]: #range(0, 540, 10):
    for pred in ["convpaint_l-0_s-1-2"]:#prediction_types:
        res = analyse_cellpose_single_file(folder_path, img_num, mode="all", bin=0.1, scribble_width=2, suff="a", pred_tag=pred, show_res=False)
        df_list.append(res)
df = pd.concat(df_list, ignore_index=True)
df[:5]


,img_num,prediction type,scribbles mode,scribbles bin,suffix,class_1_pix_gt,class_2_pix_gt,pix_labelled,class_1_pix_labelled,class_2_pix_labelled,...,perc. labelled,accuracy,mPrecision,mRecall,mIoU,mF1,image,ground truth,scribbles,prediction
0,1,convpaint_l-0_s-1-2,all,0.1,a,128463,67633,145,100,45,...,0.073943,0.884169,0.906561,0.840435,0.761474,0.861883,cellpose_train_imgs/001_img.png,cellpose_train_imgs/001_ground_truth.png,cellpose_train_imgs/001_scribbles_all_00100_w2...,cellpose_train_imgs/001_convpaint_l-0_s-1-2_al...


Save the data frame in a csv file:

In [ ]:
from datetime import datetime
time_stamp = datetime.now().strftime("%y%m%d%H%M%S")
file_name = f"test_labels_vs_acc_{time_stamp}.csv"
df.to_csv(file_name, index=False)